目标检测之YoLoV1
===

# 1.什么是YoLo
YOLO = You Only Look Once，只需要看一次，不需要类似RPN的候选框提取，直接进行整图回归就可以了

# 2.算法概述
对于输入是448或者224大小，原理其实是一样的，我们以下按照输入448大小的图片来介绍
- 将图像划分为固定的网格（比如$14 \times 14$），如果一个目标的中心落入格子，则该格子就负责检测该目标
- 每个网格预测包含2个边框盒以及20个判别属性(这是针对VOC训练集而言的，VOC有20个类别)。每个边框盒包括4个坐标以及一个置信值Confidence。所以网格里面包含的数据维度就是$2 \times 5 + 20 = 30$维
- 置信值为格子包含目标的概率与IOU的乘积
- 每个格子预测包含某一类目标的条件概率值
- 每个bounding box通过对应格子的类别概率和box置信度相乘得到该类别的置信分数。这个分数衡量了该类别出现在box中的概率以及该box与目标的吻合程度
- 网络输出层即为每个Grid的对应结果，由此实现端到端的训练。

![images](...%0AAAAAAElFTkSuQmCC)

对于上图来说，我们在Ground truth中可以找到狗的四个坐标，然后可以找到它的中心点坐标，同时我们可以将狗的中心坐标归一化到$14 \times 14$的网格中，找到它落在哪个网格上，就能知道这个网格负责预测狗。同理自行车和汽车都可以做这样的处理。

# 3.网络结构
![images](...)

上图仅仅是一个示意图。具体来说，YOLO v1一般存在两种结构：Res50和Vgg16。对于输入是$(448,448,3)$的图片来说，两种网络最终的输出都是(BS,14,14,30)，对于输入是$(224,224,3)$的图片来说，输出都是(BS,7,7,30)

# 4.数据集
对于训练数据集，我们使用的是VOC2007 train + val + VOC2012 train共16551张图片。对于训练集来说，我们需要做一些图片增强(arguments)的操作，比如随机裁剪，随机翻转，随机模糊，随机亮度调整，随机对比度调整，随机饱和度调整，随机移动等等操作，最终还要减去图片均值以及归一化操作，最后将每张图片的大小调整到$448 \times 448$，具体代码实现可以参考YOLOV1 dataset的实现。在数据集的实现中，最关键的也是最难理解的就是对于label和ground truth box的编码操作。我们接下来仔细讲解这一部分内容

## 4.1.数据集编码
从数据集中得到的每一张图片，都有两个内容，一部分是图片的信息，经过图片增强操作时候，大小为$(448,448,3)$，另一部分就是这张图片的ground truth box以及包含物体的label的信息，最终我们需要将这一部分信息，编码到一个$[14,14,30]$的矩阵中去。具体的代码如下

In [ ]:
import torch
grid_num = 14
boxes = torch.zeros()
labels = torch.zeros()
target = torch.zeros((grid_num,grid_num,2 * 5 + 20))
cell_size = 1./grid_num
wh = boxes[:,2:]-boxes[:,:2]
cxcy = (boxes[:,2:]+boxes[:,:2])/2
for i in range(cxcy.size()[0]):
    cxcy_sample = cxcy[i]
    ij = (cxcy_sample/cell_size).ceil()-1 # 计算当前格子的index坐标位置
    target[int(ij[1]),int(ij[0]),4] = 1
    target[int(ij[1]),int(ij[0]),9] = 1
    target[int(ij[1]),int(ij[0]),int(labels[i])+9] = 1
    xy = ij*cell_size #匹配到的网格的左上角相对坐标
    delta_xy = (cxcy_sample -xy)/cell_size
    target[int(ij[1]),int(ij[0]),2:4] = wh[i]
    target[int(ij[1]),int(ij[0]),:2] = delta_xy
    target[int(ij[1]),int(ij[0]),7:9] = wh[i]
    target[int(ij[1]),int(ij[0]),5:7] = delta_xy

1. 首先我们需要初始化一块$14,14,30$的空间，这个14就是整个图片需要分割成$14 \times 14$的空间，30就是$2 * 5 + 20$，5代表每个格子的四个坐标位置，以及当前格子预测物体的置信度信息，由于每个格子会生成2个预测框，所以乘以2，20就是VOC数据集包含的物体总类别，如果是COCO数据集，那就是80
2. cell_size就是每个格子归一化之后的大小，接着wh就是每个真实框的宽高，cxcy就是每个真实框的中心点坐标(有多少个真实框，就有多少个中心点坐标)
3. 接下来就是处理每个格子的编码了。首先找到当前真实框中心点在$14 \times 14$上的index坐标位置，换句话说，就是找到当前真实框由$14 \times 14$上的哪一个格子负责预测(真实框的中心点落在哪个格子上，就由哪个格子预测)，然后开始给当前格子编码，第4,9两个位置设置为1，这个原因是，这两个位置是两个预测框的置信度位置，由于这个格子上有真实框的中心点落在上面，那它肯定就负责预测这个物体了，所以置信度当然是1.然后将代表的物体进行one-hot编码，放入后20位中。
4. 最后调整坐标，使之变成$x,y,w,h$这样的格式

# 5.损失函数
我们有了网络，就有了最后的pred结果，shape就是$BS,14,14,30$，也有了真实值ground truth->targets，shape是$BS,14,14,30$，那么接下来就是损失计算了。

## 5.1.损失函数介绍
$$
\begin{eqnarray}
Loss&=&\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}[(x_i-\hat{x}_i)^2+(y_i-\hat{y}_i)^2] \\
&+&\lambda\_{coord}\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{obj}[(\sqrt{\omega_i}-\sqrt{\hat{\omega}_i})^2+(\sqrt{h_i}-\sqrt{\hat{h}_i})^2] \\
&+&\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{obj}(C_i-\hat{C}\_i)^2 \\
&+&\lambda\_{noobj}\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{noobj}(C_i-\hat{C_i})^2\\
&+&\sum\_{i=0}^{S^2}\ell\_{i}^{obj}\sum\_{c \in classes}[p_i(c)-\hat{p}_i(c)]^2
\end{eqnarray}
$$

参数有如下输入：
$$
\begin{eqnarray}
\lambda_{coord}&=&5\\
\lambda_{noobj}&=&0.5\\
S&=&14\\
B&=&2\\
C&=&20
\end{eqnarray}
$$
S就是每张图片划分成了$14 \times 14$个格子，B就是每个格子里面有2套边框，C是类别，有20个类别的物体

## 5.2.具体做法

In [ ]:
pred_tensor = None
target_tensor=None
B = 0
dataConfig = None
N = pred_tensor.size()[0]
coo_mask = target_tensor[:,:,:,4] > 0 # shape: ?,14,14
noo_mask = target_tensor[:,:,:,4] == 0 # shape: ?,14,14
coo_mask = coo_mask.unsqueeze(-1).expand_as(target_tensor)
noo_mask = noo_mask.unsqueeze(-1).expand_as(target_tensor)

coo_pred = pred_tensor[coo_mask].view(-1,B * 5 + len(dataConfig.getClasses()))
coo_target = target_tensor[coo_mask].view(-1,B * 5 + len(dataConfig.getClasses()))

由于每个格子的30位编码中，第4，9两位置表明的是置信度，也就是说，如果这两位是1，那么这个格子就负责预测真实框，否则就不负责预测真实框。所以我们一眼就能看出coo_mask就是找到$14 \times 14$这么多格子中负责预测真实框的格子的位置，它的shape就是$BS,14,14$，内容非0即1，如果原图有两个ground truth box，那么coo_mask里面就有两个位置是1，其它的都是0；noo_mask与coo_mask的意义正好相反，最后将coo_mask和noo_mask进行扩展，变为了$BS,14,14,30$的结构，原先是1的，就成了30个1，原先是0的，就成了30个0<br/>
接下来，我们找到了预测信息中对应的应该进行预测的格子的信息coo_pred,同样的找到那些真实信息中对应的进行真实框预测的格子的信息coo_target。

### 5.2.1.位置预测损失-Localization Loss(loc_loss)
$$
\begin{eqnarray}
& &\lambda\_{coord}\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{obj}[(x\_i-\hat{x}\_i)^2+(y\_i-\hat{y}\_i)^2] \\
&+&\lambda\_{coord}\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{obj}[(\sqrt{\omega\_i}-\sqrt{\hat{\omega}\_i})^2+(\sqrt{h\_i}-\sqrt{\hat{h}\_i})^2]
\end{eqnarray}
$$
$\ell_{ij}^{obj}$:如果第i个格子的第j个bbox表示这个object，那么就是1，否则就是0.$x,y,\omega,h$表示的就是bbox的坐标-左上角坐标$x,y$，宽$\omega$,长$h$

In [ ]:
box_pred = coo_pred[:, :10].reshape(-1,5)
box_target = coo_target[:,:10].contiguous().view(-1,5)

coo_response_mask     = torch.cuda.ByteTensor(box_target.size()) if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else torch.ByteTensor(box_target.size())
coo_response_mask.zero_()
coo_not_response_mask = torch.cuda.ByteTensor(box_target.size()) if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else torch.ByteTensor(box_target.size())
coo_not_response_mask.zero_()

for i in range(0,box_target.size()[0],2):
    box1 = box_pred[i:i+2]
    box1_xyxy = torch.autograd.Variable(torch.FloatTensor(box1.size()).cuda() if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else torch.FloatTensor(box1.size()))
    box1_xyxy[:,:2] = box1[:,:2] -0.5*box1[:,2:4]
    box1_xyxy[:,2:4] = box1[:,:2] +0.5*box1[:,2:4]
    box2 = box_target[i].view(-1,5)
    box2_xyxy = torch.autograd.Variable(torch.FloatTensor(box2.size()).cuda() if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else torch.FloatTensor(box2.size()))
    box2_xyxy[:,:2] = box2[:,:2] -0.5*box2[:,2:4]
    box2_xyxy[:,2:4] = box2[:,:2] +0.5*box2[:,2:4]
    iou = self.compute_iou(box1_xyxy[:,:4],box2_xyxy[:,:4]) #[2,1]
    max_iou,max_index = iou.max(0)
    max_index = max_index.data.cuda() if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else max_index.data
    coo_response_mask[i+max_index]=1
    coo_not_response_mask[i+1-max_index]=1

box_pred_response = box_pred[coo_response_mask].view(-1,5)
box_target_response = box_target[coo_response_mask].view(-1,5)

loc_loss = torch.nn.functional.mse_loss(box_pred_response[:,:2],box_target_response[:,:2],size_average=False) \
           + torch.nn.functional.mse_loss(torch.sqrt(box_pred_response[:,2:4]),torch.sqrt(box_target_response[:,2:4]),size_average=False)

由于我们找到了预测值中应该负责预测的格子的信息coo_pred以及真实值中负责预测的格子的信息coo_target，那么我们可以找到每个负责预测的格子的坐标与置信度box_pred以及box_target。这里需要注意一点，由于每个负责预测的格子有两个预测框来负责预测一个物体。所以如果一张图片上有4个ground truth box,那么生成的box_target的shape就应该是$(8,5)$，5就是(x,y,w,h,c)，那么8就是$4 \times 2$，相邻的两个是一个格子上的预测框。所以我们可以看到循环的是box_target.size，并且呢step是2。循环里面呢，可以看到首先找到当前的预测框及其兄弟预测框，然后找到对应的真实框及其兄弟真实框，然后计算IOU，找到IOU相对大的那个预测框，那么这个预测框就负责进行预测，它的兄弟预测框由于IOU较小，就不负责预测了。负责预测的放入coo_response_mask中，不负责预测的负责coo_not_response_mask中(这个值在信心损失中需要用到)<br/>
这样我们就找到了具体的负责预测的预测框的预测值与对应的ground truth box。然后两个进行MSE损失。根据论文。x,y坐标的损失就是直接做MSE，w,h的损失则是$\sqrt{w},\sqrt{h}$的MSE损失

### 5.2.2.信心预测损失-Confidence Loss
$$
\begin{eqnarray}
& &\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{obj}(C\_i-\hat{C}\_i)^2\\
&+&\lambda\_{noobj}\sum\_{i=0}^{S^2}\sum\_{j=0}^B\ell\_{ij}^{noobj}(C\_i-\hat{C\_i})^2\\
\end{eqnarray}
$$
第一项表示含object的bbox的Confidence预测；第二项表示不含object的bbox的Confidence预测

In [ ]:
contain_loss = torch.nn.functional.mse_loss(box_pred_response[:,4],box_target_response[:,4],size_average=False)

信心预测损失，也叫做置信度预测损失，包含两个内容的部分：包含bbox的置信度损失，以及不包含bbox的置信度损失。包含bbox的置信度损失很容易计算，我们只要计算box_pred_response$[:,4]$的损失就好了

In [ ]:
noo_pred = pred_tensor[noo_mask].view(-1, self.B * 5 + len(self.dataConfig.getClasses()))
noo_target = target_tensor[noo_mask].view(-1, self.B * 5 + len(self.dataConfig.getClasses()))     # 例如：[1496,30]
noo_pred_mask = torch.cuda.ByteTensor(noo_pred.size()) if self.CFG.CONFIG.USEGPU and torch.cuda.is_available() else torch.ByteTensor(noo_pred.size())  # 例如：[1496,30]
noo_pred_mask.zero_()   #初始化全为0
noo_pred_mask[:,4]=1
noo_pred_mask[:,9]=1
noo_pred_c = noo_pred[noo_pred_mask]
noo_target_c = noo_target[noo_pred_mask]
nooobj_loss = torch.nn.functional.mse_loss(noo_pred_c,noo_target_c,size_average=False)

2. 不包含物体的置信度损失：这段代码相信很容易看得懂

### 5.2.3.类别预测损失-Classification Loss

$$\sum\_{i=0}^{S^2}\ell\_{i}^{obj}\sum\_{c \in classes}[p\_i(c)-\hat{p}\_i(c)]^2$$

$\ell_{i}^{obj}$:如果第i个格子包含某个object的中心的话，那么这个值就是1，否则就是0<br/>
$p_i(c)$：当前格子所预测的类别是类别c的概率

In [ ]:
class_pred = coo_pred[:,10:]
class_target = coo_target[:,10:]
class_loss = torch.nn.functional.mse_loss(class_pred,class_target,size_average=False)

对于类别损失，由于这一步我们是做类别损失，所以只关心后20位的one-hot编码就可以了，也就是class_pred,同样的找到class_target，然后两者做MSE损失，这就是类别损失

### 5.2.4.总结

损失函数中：
- 只有当某个网格中有object的时候才对classification error进行惩罚。
- 只有当某个box predictor对某个ground truth box负责的时候，才会对box的coordinate error进行惩罚，而对哪个ground truth box负责就看其预测值和ground truth box的IoU是不是在那个cell的所有box中最大。

# 6.缺点

- 由于输出层为全连接层，因此在检测时，YOLO训练模型只支持与训练图像相同的输入分辨率。
- 虽然每个格子可以预测B个bounding box，但是最终只选择只选择IOU最高的bounding box作为物体检测输出，即每个格子最多只预测出一个物体。当物体占画面比例较小，如图像中包含畜群或鸟群时，每个格子包含多个物体，但却只能检测出其中一个。这是YOLO方法的一个缺陷。
- YOLO loss函数中，大物体IOU误差和小物体IOU误差对网络训练中loss贡献值接近（虽然采用求平方根方式，但没有根本解决问题）。因此，对于小物体，小的IOU误差也会对网络优化过程造成很大的影响，从而降低了物体检测的定位准确性。